In [1]:
# Datenverarbeitung
import pandas as pd

# Deaktivierung der Warnungen
import warnings
warnings.filterwarnings('ignore')

## Der Datensatz

Der Datensatz stammt von https://www.kaggle.com/datasets/debashish311601/commodity-prices?select=commodity_futures.csv. Er besteht aus 35448 Einträgen und hat 24 Features: 23 unterschiedliche Waren und Rohstoffe und einer Datumsspalte. Ein Eintrag stellt die Preise der Waren an einem spezifischen Tag dar. Der erfasste Zeitraum geht vom 03.01.2000 bis zum 18.11.2022. Erfasst wurden die Preise des Euro-Raums.<br>
Die erfassten Waren und Preise lassen sich in sechs Kategorien zusammenfassen:
1. Energie:          Erdgas, "West Texas Intermediate"-Öl, Brent-Öl, schwefelarmes Gasöl, Diesel, Benzin
2. Edelmetalle:      Gold, Silber
3. Industriemetall:  Kupfer, Aluminium, Zink, Nickel
4. Getreideprodukte: Sojabohnen, Mais, Sojabohnenöl, Sojabohnenschrot, Weizen, Winterweizen
5. Lebendvieh:       Rind, Schwein (als Handelsware für die Fleischproduktion)
6. "Softs":          Zucker, Kaffee, Baumwolle.<br>
Alle Preise wurden in Euro pro Maßeinheit erhoben. Die Waren wurden in unterschiedliche Maßeinheiten erhoben:
- Erdgas in Million British thermal units (MMBtu)
- "West Texas Intermediate"-Öl, Brent-Öl in Barrel
- Diesel, Benzin in Gallon
- schwefelarmes Gasöl, Aluminium, Zink, Nickel, Sojabohnenschrot in Metertonne (MT)
- Sojabohnen, Mais, Weizen, Winterweizen in Bushel
- Kupfer, Sojabohnenöl, Rind, Schwein, Zucker, Kaffee, Baumwolle in britisch Pfung (lb)
- Gold und Silber in Feinunze (oz.).

In [2]:
# Laden des Datensatzes
commodity = pd.read_csv("data/commodity prices/commodity_futures.csv")
# Erster Blick auf den Datensatz; Prüfen der Spaltenbezeichnungen
commodity.head()

,Date,NATURAL GAS,GOLD,WTI CRUDE,BRENT CRUDE,SOYBEANS,CORN,COPPER,SILVER,LOW SULPHUR GAS OIL,...,ZINC,ULS DIESEL,NICKEL,WHEAT,SUGAR,GASOLINE,COFFEE,LEAN HOGS,HRW WHEAT,COTTON
0,2000-01-03,NaN,NaN,NaN,NaN,456.50,200.75,NaN,NaN,NaN,...,1237.50,NaN,8446.0,247.50,6.10,NaN,116.50,55.975,274.25,51.07
1,2000-01-04,2.176,283.7,25.55,24.39,464.25,203.00,0.8480,5.375,213.50,...,1215.00,67.78,8314.0,247.25,5.77,NaN,116.25,55.625,274.00,50.73
2,2000-01-05,2.168,282.1,24.91,23.73,469.25,203.00,0.8565,5.210,213.00,...,1209.25,66.55,8307.0,249.75,5.81,NaN,118.60,55.075,276.25,51.56
3,2000-01-06,2.196,282.4,24.78,23.62,468.00,203.75,0.8530,5.167,211.25,...,1212.00,66.28,8252.0,248.50,5.77,NaN,116.85,55.175,275.00,52.08
4,2000-01-07,2.173,282.9,24.22,23.09,471.50,207.00,0.8540,5.195,205.25,...,1209.25,64.75,8174.0,251.75,5.84,NaN,114.15,55.625,277.75,53.96


In [3]:
# Ändern der Spaltennamen auf Kleinschreibung
commodity.columns = map(str.lower, commodity.columns)

# Ersetzen von Leerzeichen zu Unterstrichen
commodity.columns = commodity.columns.str.replace(" ", "_")
# Prüfen der neuen Spaltennamen
commodity.head()

,date,natural_gas,gold,wti_crude,brent_crude,soybeans,corn,copper,silver,low_sulphur_gas_oil,...,zinc,uls_diesel,nickel,wheat,sugar,gasoline,coffee,lean_hogs,hrw_wheat,cotton
0,2000-01-03,NaN,NaN,NaN,NaN,456.50,200.75,NaN,NaN,NaN,...,1237.50,NaN,8446.0,247.50,6.10,NaN,116.50,55.975,274.25,51.07
1,2000-01-04,2.176,283.7,25.55,24.39,464.25,203.00,0.8480,5.375,213.50,...,1215.00,67.78,8314.0,247.25,5.77,NaN,116.25,55.625,274.00,50.73
2,2000-01-05,2.168,282.1,24.91,23.73,469.25,203.00,0.8565,5.210,213.00,...,1209.25,66.55,8307.0,249.75,5.81,NaN,118.60,55.075,276.25,51.56
3,2000-01-06,2.196,282.4,24.78,23.62,468.00,203.75,0.8530,5.167,211.25,...,1212.00,66.28,8252.0,248.50,5.77,NaN,116.85,55.175,275.00,52.08
4,2000-01-07,2.173,282.9,24.22,23.09,471.50,207.00,0.8540,5.195,205.25,...,1209.25,64.75,8174.0,251.75,5.84,NaN,114.15,55.625,277.75,53.96


### Maßeinheiten

Da die unterschiedlichen Messeinheiten zu großen Verzerrungen in der graphischen Darstellung der Daten führen, werden diese etwas angeglichen. Pfund (lb) wird in Kilogram umgerechnet. (Meter)Tonne (MT) wird ebenfalls in Kilogramm, aber multipliziert mit 100 umgerechnet. Hierdurch können die Daten graphisch besser dargestellt werden. Da die Warenpreise nicht direkt untereinander verglichen werden, besteht hier nicht die Gefahr verzerrter Darstellung oder falscher Schlussfolgerungen.

In [4]:
# lb to kg
lb_to_kg = ['copper', 'soybean_oil', 'live_cattle','lean_hogs','sugar','coffee','cotton']
commodity[lb_to_kg] = (commodity[lb_to_kg] / (1/2.205))

# MT to kg*100
mt_to_kg = ['aluminium', 'zinc', 'nickel', 'soybean_meal'] 
commodity[mt_to_kg] = (commodity[mt_to_kg] / 1000)*100
commodity['copper'] = commodity['copper']*100

### Umformung des ursprünglichen Datenatzes
Da der Datensatz nur die Warenwerte, aber nicht die Kategorien erfasst, werden neue Spalten erstellt, in denen der Tagesdurchschnittswert der jeweiligen Kategorie erfasst wird.

In [5]:
# Erstellen von Spalten mit Durchschnittswerten pro Waren-Kategorie und Speicherung in ursprünglichen Datensatz
commodity['energy'] = commodity[['natural_gas','wti_crude','brent_crude','low_sulphur_gas_oil','uls_diesel','gasoline']].mean(axis=1)
commodity['precious_metals'] = commodity[['gold','silver']].mean(axis=1)
commodity['industrial_metals'] = commodity[['copper','aluminium','zinc','nickel']].mean(axis=1)
commodity['grains'] = commodity[['soybeans','corn','soybean_oil','soybean_meal','wheat','hrw_wheat']].mean(axis=1)
commodity['livestocks'] = commodity[['live_cattle','lean_hogs']].mean(axis=1)
commodity['softs'] = commodity[['sugar','coffee','cotton']].mean(axis=1)

#### Erstellen von Sub-Dataframes
Damit der Datensatz graphisch besser dargestellt werden kann, muss er umgeformt werden. Für diesen Zweck, wird für jede Kategorie ein Sub-Dataframe estellt. Zusätzlich wird eine Namensspalte für jede Kategorie angelegt. Dies wird im weiteren Verlauf nützlich.

In [6]:
# Erstellen eines neuen Sub-Dataframes für die Kategorie Energie (Erdgas, "West Texas Intermediate", Brent-Öl, schwefelarmes Gasöl, Diesel, Benzin)
df_energy = commodity[['date','energy','natural_gas','wti_crude','brent_crude','low_sulphur_gas_oil','uls_diesel','gasoline']]
# Namensvariable
df_energy['category'] = 'energy'
# Umbenennung des Durchschnittswertes der Kategorie in 'mean_category'
df_energy.rename(columns={'energy':'mean_category'}, inplace=True)

# Sub-Dataframe für Edelmetalle (Gold, Silber)
df_precious_metals = commodity[['date','precious_metals', 'gold','silver']]
df_precious_metals['category'] = 'precious_metals'
df_precious_metals.rename(columns={'precious_metals':'mean_category'}, inplace=True)

# Sub-Dataframe für Industrielle Metalle (Kupfer, Aluminium, Zink, Nickel)
df_industrial_metals = commodity[['date','industrial_metals','copper','aluminium','zinc','nickel']]
df_industrial_metals['category'] = 'industrial_metals'
df_industrial_metals.rename(columns={'industrial_metals':'mean_category'}, inplace=True)

# Sub-Dataframe für Getreide etc. (Sojabohnen, Mais, Sojabohnenöl, Sojabohnenschrot, Weizen, Winterweizen)
df_grains = commodity[['date','grains','soybeans','corn','soybean_oil','soybean_meal','wheat','hrw_wheat']]
df_grains['category'] = 'grains'
df_grains.rename(columns={'grains':'mean_category'}, inplace=True)

# Sub-Dataframe für Lebendvieh - als Handelsware für die Fleischproduktion(Rind, Schwein)
df_livestocks = commodity[['date','livestocks','live_cattle','lean_hogs']]
df_livestocks['category'] = 'livestocks'
df_livestocks.rename(columns={'livestocks':'mean_category'}, inplace=True)

# Sub-Dataframe für 'Softs' (Zucker, Kaffee, Baumwolle)
df_softs = commodity[['date','softs','sugar','coffee','cotton']]
df_softs['category'] = 'softs'
df_softs.rename(columns={'softs':'mean_category'}, inplace=True)

### Weiterverarbeitung von neuem Dataframe
Die sechs neuentstandenen Subdataframes werden via for-Schleife zu einem neuen Datensatz verarbeitet.

In [7]:
# Zusammenführen der Sub-Dataframes zu neuen Dataframe
df = pd.DataFrame(columns=['date', 'category', 'mean_category'])
data_frames = [df_energy, df_precious_metals, df_industrial_metals, df_grains, df_livestocks, df_softs]

for data_frame in data_frames:
    df = pd.merge(df,
              data_frame,
              on=['date',
                  'category',
                  'mean_category'],
              how='outer')

Die vorhandene Variable mit dem Datums-String wird ins Datumsformat umgewandelt. Zusätzlich werden die zwei weiteren Datumsvariablen 'month' und 'year' hinzugefügt.

In [8]:
# Erstellen von Datumsvariablen
df.date = pd.to_datetime(df.date)
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

# Sortierung der Spalten
df.set_index(['date', 'year', 'month', 'category'], inplace=True)
df = df.reset_index()

# Prüfung des Datensatzes
display(df.head())
display(df.shape)

,date,year,month,category,mean_category,natural_gas,wti_crude,brent_crude,low_sulphur_gas_oil,uls_diesel,...,corn,soybean_oil,soybean_meal,wheat,hrw_wheat,live_cattle,lean_hogs,sugar,coffee,cotton
0,2000-01-03,2000,1,energy,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,2000,1,energy,66.6792,2.176,25.55,24.39,213.50,67.78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,2000,1,energy,66.0716,2.168,24.91,23.73,213.00,66.55,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,2000,1,energy,65.6252,2.196,24.78,23.62,211.25,66.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,2000,1,energy,63.8966,2.173,24.22,23.09,205.25,64.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(35448, 28)

In [9]:
# speichern des df
# df.to_csv("data/commodity_df.csv", index=False)